### 두개의 투자 전략을 하나의 class 선언

1. 클래스의 이름은 Invest 상속은 X
2. 생성자 함수 
    - 매개변수 4개
        - 주식 데이터 (_df)
        - 기준이 되는 컬럼의 이름 (_col = 'Adj Close')
        - 투자의 시작 시간 (_start = '2010-01-01')
        - 투자의 종료 시간 (_end = datetime.now())
    - 결측치, 양의 무한대, 음의 무한대를 제외
    - 컬럼중 Date 컬럼이 존재한다면 Date 컬럼을 인덱스로 변환
    - index를 시계열 데이터로 변환
    - index에 tz에 존재한다면 tz를 None 변경
    - 시작 시간과 종료 시간은 시계열 데이터로 변환
        - 종료시간은 문자열인 경우에만 변환환
    - 기준이 컬럼(self.col), 시작시간(self.start), 종료시간(self.end)은 self변수로 생성
3. buyandhold 함수를 생성 
    - 매개변수 x (self를 생성)
    - 시작시간과 종료시간으로 인덱스 필터 기준이되는 컬럼을 제외하고 모두 제거 
    - 일별 수익율 컬럼을 생성하여 pct_change() + 1 함수를 이용하여 값을 대입
    - 누적수익율 컬럼을 생성하여 누적 수익율 데이터를 대입 
    - 만들어진 데이터프레임과 총 누적수익율을 되돌려준다.

In [1]:
import pandas as pd 
from datetime import datetime
import numpy as np

In [10]:
class Invest:
    # 생성자 함수 
    def __init__(
            self,
            _df, 
            _col = 'Adj Close', 
            _start = '2010-01-01', 
            _end = datetime.now()
    ):
        # 조건식 생성 
        flag = _df.isin( [np.nan, np.inf, -np.inf ] ).any(axis=1)
        self.df = _df.loc[~flag]
        # self.df의 컬럼중 Date가 존재한다면?
        if 'Date' in self.df.columns:
            self.df.set_index('Date', inplace=True)
        # index를 시계열 데이터로 변경 
        self.df.index = pd.to_datetime(self.df.index)
        # tz가 존재하는가?
        if self.df.index.tz:
            self.df.index = self.df.index.tz_localize(None)
        # 인자값으로 들어온 기준이 되는 컬럼의 이름은 self변수에 저장
        self.col = _col
        # 시작 시간은 시계열로 변경 
        try:
            self.start = datetime.strptime(_start, '%Y-%m-%d')
            if type(_end) == 'str':
                self.end = datetime.strptime(_end, '%Y-%m-%d')
            else:
                self.end = _end
        except:
            print("시간의 타입이 맞지 않습니다. 포멧은 YYYY-mm-dd 형식입니다.")
            print('Class를 다시 생성하시기 바랍니다.')
        
    def buyandhold(self):
        # 시작 시간, 종료시간 , 특정 컬럼을 기준으로 데이터를 필터 
        result = self.df.loc[self.start : self.end , [self.col]]
        # 일별 수익율 생성
        result['rtn'] = (result[self.col].pct_change() + 1).fillna(1)
        # 누적 수익율 생성 
        result['acc_rtn'] = result['rtn'].cumprod()
        # 총 누적 수익율을 변수에 저장 
        acc_rtn  = result.iloc[-1, -1]
        return result, acc_rtn


In [3]:
df = pd.read_csv('../../csv/AAPL.csv')

In [11]:
test_invest = Invest(df)

In [12]:
test_invest.buyandhold()

(             Adj Close       rtn   acc_rtn
 Date                                      
 2010-01-04   26.782711  1.000000  1.000000
 2010-01-05   26.829010  1.001729  1.001729
 2010-01-06   26.402260  0.984094  0.985795
 2010-01-07   26.353460  0.998152  0.983973
 2010-01-08   26.528664  1.006648  0.990515
 ...                ...       ...       ...
 2019-06-18  198.449997  1.023518  7.409631
 2019-06-19  197.869995  0.997077  7.387975
 2019-06-20  199.460007  1.008036  7.447342
 2019-06-21  198.779999  0.996591  7.421952
 2019-06-24  199.169998  1.001962  7.436514
 
 [2384 rows x 3 columns],
 7.436513727083075)